**Import Libraries**

In [107]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
from sklearn.linear_model import LogisticRegression

**Read data**

In [108]:
train = pd.read_csv(r"C:\Users\DSN\Downloads\playground-series-s4e7\train.csv")
test =  pd.read_csv(r"C:\Users\DSN\Downloads\playground-series-s4e7\test.csv")
train.head()

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
0,0,Male,21,1,35.0,0,1-2 Year,Yes,65101.0,124.0,187,0
1,1,Male,43,1,28.0,0,> 2 Years,Yes,58911.0,26.0,288,1
2,2,Female,25,1,14.0,1,< 1 Year,No,38043.0,152.0,254,0
3,3,Female,35,1,1.0,0,1-2 Year,Yes,2630.0,156.0,76,0
4,4,Female,36,1,15.0,1,1-2 Year,No,31951.0,152.0,294,0


In [109]:
test_id =test['id']

***Running Data Diagoistics***\
**Data Shape**

In [110]:
#check the shape
train.shape

(11504798, 12)

In [111]:
#check columns information
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11504798 entries, 0 to 11504797
Data columns (total 12 columns):
 #   Column                Dtype  
---  ------                -----  
 0   id                    int64  
 1   Gender                object 
 2   Age                   int64  
 3   Driving_License       int64  
 4   Region_Code           float64
 5   Previously_Insured    int64  
 6   Vehicle_Age           object 
 7   Vehicle_Damage        object 
 8   Annual_Premium        float64
 9   Policy_Sales_Channel  float64
 10  Vintage               int64  
 11  Response              int64  
dtypes: float64(3), int64(6), object(3)
memory usage: 1.0+ GB


**Missing values**

In [112]:
#missing values
# train.isnull().sum()

**Check Duplicates**

In [113]:
#check duplicates
#train.duplicated().sum()

**Check Unique Categorical Data**

In [114]:
for i in train.select_dtypes(include = "object").columns:
    print (train[i].value_counts())
    print()

Gender
Male      6228134
Female    5276664
Name: count, dtype: int64

Vehicle_Age
1-2 Year     5982678
< 1 Year     5044145
> 2 Years     477975
Name: count, dtype: int64

Vehicle_Damage
Yes    5783229
No     5721569
Name: count, dtype: int64



**Exploratory Data Analysis**

In [115]:
#train.describe()

In [116]:
#train.describe(include= object)

In [117]:
#sns.pairplot(train.iloc[:,1:5])

In [118]:

"""for column in train.select_dtypes(include ="number").columns:
    sns.histplot(data= train, x = column)
    plt.figure(figsize =(5,5))
    plt.show()"""

'for column in train.select_dtypes(include ="number").columns:\n    sns.histplot(data= train, x = column)\n    plt.figure(figsize =(5,5))\n    plt.show()'

In [119]:
# check outliers using boxplot

#for column in data.select_dtypes(include ="number").columns:
#    sns.boxplot(data= train, x = column)
#  plt.figure(figsize =(2,3))
#    plt.show()

In [120]:
# to understand relationship using scatter plot with the target variable
 

train.select_dtypes(include= 'number').columns

Index(['id', 'Age', 'Driving_License', 'Region_Code', 'Previously_Insured',
       'Annual_Premium', 'Policy_Sales_Channel', 'Vintage', 'Response'],
      dtype='object')

In [121]:
""" 
for column in ['id', 'Age', 'Driving_License', 'Region_Code', 'Previously_Insured',
       'Annual_Premium', 'Policy_Sales_Channel', 'Vintage', ]:
    sns.scatterplot(data = train, x= column, y ='Response')
    plt.figure(figsize =(2,3))
    plt.show()
    """

" \nfor column in ['id', 'Age', 'Driving_License', 'Region_Code', 'Previously_Insured',\n       'Annual_Premium', 'Policy_Sales_Channel', 'Vintage', ]:\n    sns.scatterplot(data = train, x= column, y ='Response')\n    plt.figure(figsize =(2,3))\n    plt.show()\n    "

In [122]:
# check correlation

s = train.select_dtypes(include ='number').corr()

In [123]:
""" plt.figure( figsize=(15,15))
sns.heatmap(s, annot=True)"""

' plt.figure( figsize=(15,15))\nsns.heatmap(s, annot=True)'

In [129]:
def clean(data):

    def outlier(col):
        q1,q3 = np.percentile(col,[25,75])
        inter_q =q3-q1
        lb = q1 - 1.5 * inter_q
        ub = q3 + 1.5 * inter_q
        return lb, ub
    
        outlier(data['Annual_Premium'])
        lb,ub = outlier( data['Annual_Premium'])
        data['Annual_Premium'] = np.where(data['Annual_Premium'] < lb, lb, data['Annual_Premium'])
        data['Annual_Premium'] = np.where(data['Annual_Premium']>ub, ub,data['Annual_Premium'])

    data = data.drop(["id", 'Driving_License','Policy_Sales_Channel'], axis = 1)

    cat_column = ['Gender','Vehicle_Damage']
    

    for col in cat_column:
        le= LabelEncoder()
        data[col] = le.fit_transform(data[col])
        
    one_hot = pd.get_dummies(data=data['Vehicle_Age'], dtype=int)
    data = data.join(one_hot, on= None,  how ='left')
    data = data.drop('Vehicle_Age', axis= 1)

    from sklearn.preprocessing import StandardScaler
    sc = StandardScaler()

    scale_col = ['Age', 'Region_Code','Annual_Premium', 'Vintage']
    for cols in scale_col:
        data[cols] = sc.fit_transform(data[cols].values.reshape(-1, 1))
    return data
  
train = clean(train)
test = clean(test)

In [130]:
test

,Gender,Age,Region_Code,Previously_Insured,Vehicle_Damage,Annual_Premium,Vintage,1-2 Year,< 1 Year,> 2 Years
0,0,-1.226132,1.583259,0,0,-1.692555,0.801411,0,1,0
1,1,0.573928,0.121083,0,1,0.426701,-0.511344,1,0,0
2,1,0.573928,1.275433,0,1,-1.692555,1.339016,1,0,0
3,0,-1.092794,1.583259,1,0,-0.362616,-0.611364,0,1,0
4,1,0.840603,-0.571527,0,0,0.221908,-0.198783,1,0,0
...,...,...,...,...,...,...,...,...,...,...
7669861,1,1.240616,0.121083,0,1,1.288803,-0.686378,1,0,0
7669862,1,-0.692781,1.814129,1,0,-0.292750,0.251304,0,1,0
7669863,1,0.573928,0.505866,1,0,-1.692555,-1.261490,1,0,0
7669864,1,-0.559443,0.121083,0,1,0.510796,-0.561354,0,1,0


**Split Data**

In [132]:
X = train[['Gender', 'Age', 'Region_Code', 'Previously_Insured',
       'Vehicle_Damage', 'Annual_Premium', 'Vintage',
       '1-2 Year', '< 1 Year', '> 2 Years']]
y = train['Response'].values

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2)

In [133]:
def models(X_train, y_train):
  from sklearn.linear_model import LogisticRegression
  log = LogisticRegression(random_state = 0)
  log.fit(X_train,y_train)

  from sklearn.tree import DecisionTreeClassifier
  tree = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
  tree.fit(X_train, y_train)

  from sklearn.ensemble import RandomForestClassifier
  forest = RandomForestClassifier(n_estimators =  10, criterion = 'entropy', random_state = 0)
  forest.fit(X_train, y_train)


  print("The accuracy of Logistic Regression",log.score(X_train,y_train))
  print("The accuracy of Decision Tree",tree.score(X_train,y_train))
  print("The accuracy of Random Forest ",forest.score(X_train,y_train))

  return log,tree, forest 

In [134]:
model = models(X_train , y_train)

The accuracy of Logistic Regression 0.8769306891320773
The accuracy of Decision Tree 0.9903880316015993
The accuracy of Random Forest  0.9772818687160726


In [135]:
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

for i in range(len(model)):
  print('Model:', i)
  print(classification_report(y_test, model[i].predict(X_test)))
  print(accuracy_score(y_test, model[i].predict(X_test)))

Model: 0
              precision    recall  f1-score   support

           0       0.88      1.00      0.93   2018377
           1       0.29      0.00      0.00    282583

    accuracy                           0.88   2300960
   macro avg       0.58      0.50      0.47   2300960
weighted avg       0.81      0.88      0.82   2300960

0.8771573604060914
Model: 1
              precision    recall  f1-score   support

           0       0.91      0.91      0.91   2018377
           1       0.33      0.32      0.32    282583

    accuracy                           0.83   2300960
   macro avg       0.62      0.61      0.61   2300960
weighted avg       0.83      0.83      0.83   2300960

0.8347011682080523
Model: 2
              precision    recall  f1-score   support

           0       0.90      0.95      0.92   2018377
           1       0.36      0.21      0.27    282583

    accuracy                           0.86   2300960
   macro avg       0.63      0.58      0.59   2300960
weighted 

In [137]:
importances = model[1].feature_importances_

# Select features with importance greater than a threshold
threshold = 0.05  # Adjust as needed
selected_features = X.columns[importances > threshold]

In [138]:
selected_features

Index(['Age', 'Region_Code', 'Vehicle_Damage', 'Annual_Premium', 'Vintage'], dtype='object')

In [140]:
submission_pred_n = model[0].predict(test)

submission_pred_n

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [141]:
df =pd.DataFrame({"id": test_id.values,
                  "Response": submission_pred_n
                  })

df.to_csv("submission3.csv", index = False)